# The Results of the Baseline and Adversarially Trained Model

The code block provided is designed to validate the training process by loading the model weights of epoch 10. This validation applies to both the baseline model, which has been trained on standard data, and the model that has been specifically trained using adversarial examples. To assess the models' performance, we will conduct tests using benign test cases. For this evaluation, we will randomly select 150 samples from the benign dataset. This sample size is chosen to match the number of adversarial examples used in previous testing, ensuring a balanced comparison between the two types of models. 

In [1]:
import torch
import os
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from collections import namedtuple
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import hashlib
import json
import random

random.seed(42) # Setting seed

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
codebert_model = AutoModel.from_pretrained("microsoft/codebert-base")

batch_size = 8
num_epochs = 10
learning_rate = 1e-5
weight_decay = 0.05
class_weights = torch.tensor([1.9, 0.68])

# Create the classification head
class CodeBERTForVulnDetection(nn.Module):
  def __init__(self, codebert):
    super(CodeBERTForVulnDetection, self).__init__()

    self.codebert = codebert
    self.dropout1 = nn.Dropout(p=0.2)
    self.linear1 = nn.Linear(768, 3072)
    self.tanh = nn.Tanh()
    self.dropout2 = nn.Dropout(p=0.2)
    self.linear2 = nn.Linear(3072, 3072)
    self.classifier = nn.Linear(3072, 2)

    self.loss_func = nn.BCEWithLogitsLoss(weight=class_weights)

  def forward(self, input_ids, attention_mask):
    # codeBERT output pooled
    output = self.codebert(input_ids = input_ids, attention_mask = attention_mask)
    output = torch.mean(output.last_hidden_state, 1)

    # classification head (https://arxiv.org/pdf/2204.03214.pdf, Table 4)
    output = self.dropout1(output)
    output = self.linear1(output)
    output = self.tanh(output)
    output = self.dropout2(output)
    output = self.linear2(output)
    output = self.classifier(output)

    return output

# remove duplicates and conflicting labels
def clean_data(text_samples, labels):
  hashmap = {}
  blacklist = []

  for i in range(len(text_samples)):
    sample_digest = hashlib.sha256(text_samples[i].encode('utf-8')).hexdigest()

    if hashmap.get(sample_digest) == None:
      hashmap[sample_digest] = (labels[i], i)
    else:
      if hashmap[sample_digest][0] != labels[i] and (sample_digest not in blacklist):
        blacklist.append(sample_digest)

  for blacklisted_sample in blacklist:
    hashmap.pop(blacklisted_sample)

  values = hashmap.values()

  cleaned_samples = [text_samples[val[1]] for val in values]
  cleaned_labels = [labels[val[1]] for val in values]

  return cleaned_samples, cleaned_labels

class VulDeePeckerDataset(Dataset):
  def __init__(self, samples, labels, tokenizer):
    self.tokenizer = tokenizer
    self.samples = samples
    self.labels = labels

    # Clean Data
    self.samples, self.labels = clean_data(self.samples, self.labels)

    # Tokenize samples
    self.samples = self.tokenizer(self.samples, padding=True, truncation=True, return_tensors='pt', max_length=512)

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    label = torch.tensor([self.labels[idx], 1 - self.labels[idx]], dtype=torch.float32)
    return self.samples["input_ids"][idx].squeeze(), self.samples["attention_mask"][idx].squeeze(), label

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
def testing(checkpoint_path, adv_testing):
  # Import the dataset
  print("Loading dataset...")
  samples = []
  labels = []

  if adv_testing:
    json_directory = "ADV_data" # Directory containing the adversarial data

    for filename in os.listdir(json_directory):
        if filename.endswith(".json"): 
            file_path = os.path.join(json_directory, filename)
            with open(file_path, "r") as f:
                for line in f:
                    json_object = json.loads(line)
                    samples.append(json_object['func'])
                    labels.append(json_object['target'])
  else:
    # Directory containing the benign data
    json_directory = "test_data"

    for filename in os.listdir(json_directory):
        if filename.endswith(".json"):  
            file_path = os.path.join(json_directory, filename)
            with open(file_path, "r") as f:
                for line in f:
                    json_object = json.loads(line)
                    samples.append(json_object['func'])
                    labels.append(json_object['target'])

    num_indices = 150
    random_indices = random.sample(range(len(samples)), num_indices)

    # Get the corresponding samples using the random indices to match the number of adversarial samples
    samples = [samples[i] for i in random_indices]
    labels = [labels[i] for i in random_indices]

  # Load model architecture
  model = CodeBERTForVulnDetection(codebert_model)
  optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = model.to(device)

  # Load the test dataset
  test_dataset = VulDeePeckerDataset(samples, labels, tokenizer)
  test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  # Define evaluation function
  # Evaluate model
  def evaluate(dataloader):
    model.eval()

    all_labels = []
    all_predictions = []

    with torch.no_grad():
      for input_ids, attention_mask, labels in dataloader:
          # transfer to GPU if available
          input_ids = input_ids.to(device)
          attention_mask = attention_mask.to(device)
          labels = labels.to(device)

          # forward pass
          outputs = model.forward(input_ids, attention_mask)

          # find predictions and truths to calculate confusion matrix
          all_labels += torch.min(labels.cpu(), dim=1).indices.tolist()
          all_predictions += torch.min(outputs.cpu(), dim=1).indices.tolist()

    # evaluation scores
    cm = confusion_matrix(all_labels, all_predictions)
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions)

    # Calculate false positive rate (FPR) and false negative rate (FNR)
    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn)
    fnr = fn / (fn + tp)

    # Create a named tuple to return values
    RetType = namedtuple("RetType", ["accuracy", "precision", "recall", "f1_score", "fpr", "fnr"])

    return RetType(accuracy, precision, recall, f1, fpr, fnr)

  # Load checkpoint
  model.load_state_dict(torch.load(checkpoint_path))
  model.to(device)

  # Run evaluation
  eval_result = evaluate(test_dataloader)
  print("\rEpoch {} performance metrics:\nAccuracy: {:.4f}\nPrecision: {:.4f}\nRecall: {:.4f}\nF1 Score: {:.4f}\nFalse Positive Rate: {:.4f}\nFalse Negative Rate: {:.4f}".format(
          10,
          eval_result.accuracy,
          eval_result.precision,
          eval_result.recall,
          eval_result.f1_score,
          eval_result.fpr,
          eval_result.fnr
      ))

Conduct tests on the benign model using two distinct sets of test cases: the benign test cases and the adversarial test cases. 

In [3]:
testing('epoch10_checkpoint.ckpt', False)
print("---------------------------------------------------")
testing('epoch10_checkpoint.ckpt', True)

Loading dataset...


<ipython-input-2-747bdb743540>:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


Epoch 10 performance metrics:
Accuracy: 0.9133
Precision: 0.9302
Recall: 0.8000
F1 Score: 0.8602
False Positive Rate: 0.0300
False Negative Rate: 0.2000
---------------------------------------------------
Loading dataset...


<ipython-input-2-747bdb743540>:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))
<ipython-input-2-747bdb743540>:83: Runtim

Epoch 10 performance metrics:
Accuracy: 0.2671
Precision: 1.0000
Recall: 0.2671
F1 Score: 0.4216
False Positive Rate: nan
False Negative Rate: 0.7329


Conduct tests on the adversarially trained model using two sets of test cases: the benign test cases and the adversarial test cases. 

In [4]:
testing('epoch10_checkpoint_adv.ckpt', False)
print("---------------------------------------------------")
testing('epoch10_checkpoint_adv.ckpt', True)

Loading dataset...


<ipython-input-2-747bdb743540>:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


Epoch 10 performance metrics:
Accuracy: 0.9067
Precision: 0.9130
Recall: 0.8077
F1 Score: 0.8571
False Positive Rate: 0.0408
False Negative Rate: 0.1923
---------------------------------------------------
Loading dataset...


<ipython-input-2-747bdb743540>:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


Epoch 10 performance metrics:
Accuracy: 0.5960
Precision: 1.0000
Recall: 0.5960
F1 Score: 0.7469
False Positive Rate: nan
False Negative Rate: 0.4040


<ipython-input-2-747bdb743540>:83: RuntimeWarning: invalid value encountered in scalar divide
  fpr = fp / (fp + tn)
